- закончила с xml, обработала csv и xlsx
- изучила данные на предмет пропусков, ошибок, типов данных...
    - снова поправила паука и переделала таблицы в postgre
- следющим шагом пропишу связи таблиц в postgresql - и начну составлять презентацию

In [2]:
import psycopg2
import os
import pandas as pd
import pickle

from lxml import etree
from psycopg2 import sql
from pymongo import MongoClient

#### - Сведения о налоговых правонарушениях и мерах ответственности за их совершение

https://www.nalog.gov.ru/opendata/7707329152-taxoffence/

Сведения указываются в отношении налоговых правонарушений, решения о привлечении к ответственности за совершение которых вступили в силу в период с 1 января по 31 декабря года, предшествующего году размещения указанных сведений на сайте ФНС России, при неуплате штрафа в срок до 1 октября года размещения таких сведений на сайте ФНС России

- все ИННЮЛ и по 1 нарушению для каждого

забираем:

- Документ:
    - атрибут ДатаСост - Дата, по состоянию на которую, подготовлены данные для публикации
    - Объект СведНП
        - атрибут ИННЮЛ
    - Объект СведНаруш
        - атрибут СумШтраф - привести к float

In [3]:
def get_val(doc):
    """
    Принимает один документ
    Возвращает необходимые поля, упакованые в кортеж
    """
    val = []
    # инн = ключ
    val.append(clean(doc.find('СведНП').attrib['ИННЮЛ']))
    # дата, по состоянию на которую есть данные
    val.append(clean(doc.attrib['ДатаСост']))
    # сумма штрафа
    val.append(to_float(doc.find('СведНаруш').attrib['СумШтраф']))
    
    return tuple(val)

- столбцы таблицы:

        inn
        offense_date - дата состояния, а не нарушения!
        fine_amount

In [5]:
tax_offense_sql = 'CREATE TABLE tax_offense (\
inn text PRIMARY KEY, \
offense_date date, \
fine_amount decimal \
);'

In [ ]:
connect = psycopg2.connect(dbname="fns_data", user="user", password='', host="localhost", port=5432)
with connect:
    with connect.cursor() as cur:
        cur.execute(tax_offense_sql)

connect.close()

In [ ]:
path = 'raw_data/tax_offense/data'
xpath = "Документ"
connect = psycopg2.connect(dbname="fns_data", user="user", password='', host="localhost", port=5432)
table_name = 'tax_offense'
table_columns = ['inn', 'offense_date', 'fine_amount']

- Wall time: 4min 24s
- COUNT(*) 129 827
- DISTINCT offense_date 2021-12-31 - те одна дата

In [ ]:
# %%time
# insert_to_pgsql(path, xpath, connect, table_name, table_columns)

In [ ]:
connect.close()

### дополнительные данные

#### - Юридические лица, в состав исполнительных органов которых входят дисквалифицированные лица

https://www.nalog.gov.ru/opendata/7707329152-disqualifiedpersons/

- csv на около 5 тыс строчек: - те в общем масштабе их мало
    - Полное наименование
    - ОГРН
    - ИНН
    - КПП
    - Адрес 

- беру только список инн (здесь только уникальные значения), будет флагом

In [9]:
with_d_person = pd.read_csv('../raw_data/with_disqual_person.csv', sep=';', dtype={'G3': str})
with_d_person.head(3)

,G1,G2,G3,G4,G5
0,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""КАРТЕН""",1175749002263,5751060080,575101001,"302030,ОРЛОВСКАЯ ОБЛАСТЬ,,ОРЁЛ ГОРОД,,МОСКОВСК..."
1,"ЗАКРЫТОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО ""ВЕПРЬ""",1087205000541,7222018450,722201001,"627500,ТЮМЕНСКАЯ ОБЛАСТЬ,СОРОКИНСКИЙ РАЙОН,,БО..."
2,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""САМАН""",1190280053732,0272913945,27201001,"450095,БАШКОРТОСТАН РЕСПУБЛИКА,,УФА ГОРОД,,МАЙ..."


In [10]:
inn_disqual = tuple(with_d_person.G3)
len(inn_disqual)

4916

#### - Сведения о ценах и объемах реализации продуктовых товаров и горюче-смазочных материалов в субъектах Российской Федерации по данным контрольно-кассовой техники.

https://www.nalog.gov.ru/opendata/7707329152-fnsprice/


- можно соединять по регионам: стоимость потребительской корзины и стоимость gsm
- за периоды 2019, 2020, 2021 - берём среднее за год

- в описании структуры файла немного перепутаны столбцы (дата и код региона)...

    - GA - код региона (1, 2, 3...) -в кодировке ФНС России
    - G2 - Дата наблюдения
    - G35 - Стоимость потребительской корзины для трудоспособных граждан
    - G36 - Стоимость потребительской корзины для пенсионеров
    - G37 - Стоимость потребительской корзины для детей до 18 лет
    - G70 - Бензин марки АИ-92, руб/литр
    - G71 - Бензин марки АИ-95, руб/литр
    - G72 - Бензин марки АИ-98, руб/литр
    - G73 - Дизельное топливо, руб/литр
    - G74 - Бензин марки АИ-92, (Объем проданной продукции)
    - G75 - Бензин марки АИ-95, (Объем проданной продукции)
    - G76 - Бензин марки АИ-98, (Объем проданной продукции)
    - G77 - Дизельное топливо, (Объем проданной продукции)

In [12]:
data = pd.read_csv('../raw_data/prices_by_regions/data-20220331-structure-20220331.csv', sep=';')

In [13]:
# отбираем и переименовываем столбцы
groc_and_fuels = data[['GA', 'G2', 'G35', 'G36', 'G37', 'G70', 'G71', 'G72', 'G73', 'G74', 
                       'G75', 'G76', 'G77']]
groc_and_fuels.columns = ['region_id', 'observ_date', 'cons_basket_working_age', 
                         'cons_basket_pens', 'cons_basket_child', 'AI92_cost', 'AI95_cost', 
                         "AI98_cost", 'diesel_cost', 'AI92_volume', 'AI95_volume', 
                         "AI98_volume", 'diesel_volume']

In [14]:
groc_and_fuels.head(3)

,region_id,observ_date,cons_basket_working_age,cons_basket_pens,cons_basket_child,AI92_cost,AI95_cost,AI98_cost,diesel_cost,AI92_volume,AI95_volume,AI98_volume,diesel_volume
0,1,01.01.2019,"5023,51","4301,36","5001,97","42,96","46,87","50,2","45,53","103778,69","56968,84","121,19","13973,49"
1,1,02.01.2019,"5086,32",4354,"5064,53","42,98","46,92","50,3","45,38","185820,84","119512,26","95,09","39283,9"
2,1,03.01.2019,"4930,56","4214,3","4911,48","41,13","46,86","50,77","45,46","219913,38","126519,19","107,82","45498,67"


In [15]:
# все цены распознаёт как строку
groc_and_fuels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100810 entries, 0 to 100809
Data columns (total 13 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   region_id                100810 non-null  int64 
 1   observ_date              100810 non-null  object
 2   cons_basket_working_age  100810 non-null  object
 3   cons_basket_pens         100810 non-null  object
 4   cons_basket_child        100810 non-null  object
 5   AI92_cost                100810 non-null  object
 6   AI95_cost                100810 non-null  object
 7   AI98_cost                100810 non-null  object
 8   diesel_cost              100810 non-null  object
 9   AI92_volume              100810 non-null  object
 10  AI95_volume              100810 non-null  object
 11  AI98_volume              100810 non-null  object
 12  diesel_volume            100810 non-null  object
dtypes: int64(1), object(12)
memory usage: 10.0+ MB


In [16]:
# заменяем запятую на точку и приводим к float
for col in groc_and_fuels.columns[2:]:
    groc_and_fuels[col] = groc_and_fuels[col].apply(lambda x: x.replace(',', '.')).astype(float)

# добавляем год (чтобы усреднить за период)
groc_and_fuels['observ_year'] = pd.to_datetime(groc_and_fuels['observ_date']).dt.year
groc_and_fuels.head(3)

C:\Users\013\AppData\Local\Temp/ipykernel_11972/571331203.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  groc_and_fuels[col] = groc_and_fuels[col].apply(lambda x: x.replace(',', '.')).astype(float)
C:\Users\013\AppData\Local\Temp/ipykernel_11972/571331203.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  groc_and_fuels['observ_year'] = pd.to_datetime(groc_and_fuels['observ_date']).dt.year


,region_id,observ_date,cons_basket_working_age,cons_basket_pens,cons_basket_child,AI92_cost,AI95_cost,AI98_cost,diesel_cost,AI92_volume,AI95_volume,AI98_volume,diesel_volume,observ_year
0,1,01.01.2019,5023.51,4301.36,5001.97,42.96,46.87,50.20,45.53,103778.69,56968.84,121.19,13973.49,2019
1,1,02.01.2019,5086.32,4354.00,5064.53,42.98,46.92,50.30,45.38,185820.84,119512.26,95.09,39283.90,2019
2,1,03.01.2019,4930.56,4214.30,4911.48,41.13,46.86,50.77,45.46,219913.38,126519.19,107.82,45498.67,2019


In [17]:
groc_and_fuels.observ_year.unique()

array([2019, 2020, 2021, 2022], dtype=int64)

In [18]:
groc_and_fuels.describe()

,region_id,cons_basket_working_age,cons_basket_pens,cons_basket_child,AI92_cost,AI95_cost,AI98_cost,diesel_cost,AI92_volume,AI95_volume,AI98_volume,diesel_volume,observ_year
count,100810.000000,100810.000000,100810.000000,100810.000000,100810.000000,100810.000000,100810.000000,100810.000000,1.008100e+05,1.008100e+05,100810.000000,1.008100e+05,100810.000000
mean,43.388235,5727.310486,4878.928747,5748.071100,43.652529,47.168287,40.781701,48.460662,5.434237e+05,4.326813e+05,5607.199239,1.631631e+05,2020.151771
std,25.201476,1374.112292,1161.590332,1474.763687,4.806918,4.936959,29.591852,6.595098,6.217567e+05,7.020778e+05,14599.542048,2.647617e+05,0.946589
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,2019.000000
25%,22.000000,4953.200000,4225.492500,4963.042500,41.840000,45.280000,46.060000,46.390000,1.944471e+05,1.287709e+05,36.740000,3.815213e+04,2019.000000
50%,43.000000,5493.475000,4680.695000,5481.580000,43.270000,46.910000,50.380000,48.020000,3.439578e+05,2.297549e+05,928.335000,8.911888e+04,2020.000000
75%,64.000000,6206.660000,5282.115000,6162.002500,45.440000,49.080000,53.970000,50.920000,6.772496e+05,4.597270e+05,4798.805000,1.751593e+05,2021.000000
max,92.000000,20292.650000,16722.400000,19410.610000,60.740000,63.000000,1000.000000,70.470000,5.867661e+06,7.963218e+06,293081.090000,2.965140e+06,2022.000000


- есть наблюдения с нулевыми значениями - это скорее всего должны быть None
- сперва заменяю нули на Nan, а потом агрегирую (mean игнорирует Nan при расчёте среднего)

In [19]:
groc_and_fuels = groc_and_fuels.replace(0, None)

In [21]:
avg_groc_and_fuels = groc_and_fuels.groupby(['region_id', 'observ_year'], as_index=False)[['cons_basket_working_age', 
                         'cons_basket_pens', 'cons_basket_child', 'AI92_cost', 'AI95_cost', 
                         "AI98_cost", 'diesel_cost', 'AI92_volume', 'AI95_volume', 
                         "AI98_volume", 'diesel_volume']].mean()
avg_groc_and_fuels.head(3)

,region_id,observ_year,cons_basket_working_age,cons_basket_pens,cons_basket_child,AI92_cost,AI95_cost,AI98_cost,diesel_cost,AI92_volume,AI95_volume,AI98_volume,diesel_volume
0,1,2019,5005.002466,4269.761836,4923.843233,43.332219,48.051562,50.837918,45.404822,221138.361425,122866.607068,159.544000,65073.028384
1,1,2020,5146.416749,4387.573552,5092.815164,43.547923,48.533852,52.076913,46.012650,187478.847978,123215.335929,72.161475,60822.033087
2,1,2021,5914.448685,5031.818411,5815.689205,44.553863,50.663260,54.583315,47.691973,208157.641918,142313.449507,1248.010493,69281.618521


In [22]:
avg_groc_and_fuels.describe()

,region_id,observ_year,cons_basket_working_age,cons_basket_pens,cons_basket_child,AI92_cost,AI95_cost,AI98_cost,diesel_cost,AI92_volume,AI95_volume,AI98_volume,diesel_volume
count,340.000000,340.000000,340.000000,340.000000,340.000000,340.000000,340.000000,340.000000,340.000000,3.400000e+02,3.400000e+02,340.000000,3.400000e+02
mean,43.388235,2020.500000,6015.032872,5122.777485,6036.548506,44.703428,48.314539,58.033460,50.217531,5.254999e+05,4.289553e+05,7487.232891,1.651926e+05
std,25.238494,1.119682,1303.370487,1100.238105,1419.839363,3.459869,3.419028,27.020610,4.215236,5.988471e+05,6.889907e+05,17352.852778,2.588695e+05
min,1.000000,2019.000000,4248.657425,3623.140493,4258.991534,30.757459,42.680986,40.564110,42.806658,2.524374e+03,1.060030e+03,3.230874,7.202673e+02
25%,22.000000,2019.750000,5063.237383,4320.612277,5086.709206,42.195976,45.738340,50.108363,46.782692,1.874805e+05,1.304321e+05,748.641602,4.623857e+04
50%,43.000000,2020.500000,5804.088227,4943.945219,5736.621575,44.311764,47.980808,53.768740,49.175219,3.360336e+05,2.271063e+05,2460.292753,9.344334e+04
75%,64.000000,2021.250000,6508.271664,5532.384874,6447.250315,46.511551,50.319172,58.316250,52.444556,6.592407e+05,4.450006e+05,6820.945889,1.808378e+05
max,92.000000,2022.000000,14021.350778,11865.518556,15757.352444,60.320110,63.000000,361.476192,65.622111,4.283245e+06,5.545636e+06,195210.374889,1.936039e+06


- посмотрим совпадения кодов регионов в csv и в postgresql (поле, по которому будет соединение)

In [23]:
groc_and_fuels.region_id.unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 83, 86, 87, 89, 91, 92],
      dtype=int64)

In [25]:
# уникальные из postgre
connect = psycopg2.connect(dbname="fns_data", user="user", password='', host="localhost", port=5432)

In [31]:
with connect:
    with connect.cursor() as cur:
        select = 'SELECT DISTINCT region_code FROM reestr_msp ORDER BY region_code;'
        cur.execute(select)
        print(cur.fetchall())

[(0,), (1,), (2,), (3,), (4,), (5,), (6,), (7,), (8,), (9,), (10,), (11,), (12,), (13,), (14,), (15,), (16,), (17,), (18,), (19,), (20,), (21,), (22,), (23,), (24,), (25,), (26,), (27,), (28,), (29,), (30,), (31,), (32,), (33,), (34,), (35,), (36,), (37,), (38,), (39,), (40,), (41,), (42,), (43,), (44,), (45,), (46,), (47,), (48,), (49,), (50,), (51,), (52,), (53,), (54,), (55,), (56,), (57,), (58,), (59,), (60,), (61,), (62,), (63,), (64,), (65,), (66,), (67,), (68,), (69,), (70,), (71,), (72,), (73,), (74,), (75,), (76,), (77,), (78,), (79,), (83,), (86,), (87,), (89,), (90,), (91,), (92,), (93,), (94,), (95,)]


In [32]:
connect.close()

- код 93 в postgre - Краснодарский край, соответствует заменить на 23 в groc_and_fuels),
- 94 в postgre (Территории, находящиеся за пределами РФ и обслуживаемые Управлением режимных объектов МВД России, Байконур), в groc_and_fuels данных нет
- 95 - Чеченская Республика - соответствует код 20 в groc_and_fuels

- интересный код 0: в postgre ему соответствуют 23 inn, ни для одного из которых нет отчётов в открытых данных (на bo.nalog)

#### - Таблица 5. Численность населения России, федеральных округов, субъектов Российской Федерации, городских округов, муниципальных районов, муниципальных округов, городских и сельских поселений, городских населенных пунктов, сельских населенных пунктов с населением 3000 человек и более

https://rosstat.gov.ru/vpn_popul

https://rosstat.gov.ru/storage/mediabank/tab-5_VPN-2020.xlsx

- здесь соединять придётся по наименованию
- в таблице - данные по населённым пунктам с численностью более 3000 чел
- таблица неудобная для извлечения - наименования разноформатны
- как оказалось, данные по адресу что в MongoDb, что в xml - не очень полные и не очень корректные: перепутан уровень (город в поле региона - особенно для больших городов, село в поле города и т.д.), достаточно много пропусков, есть адреса, которые сложно интерпретировать с точки зрения численности населения (50-й километр какого-нибудь шоссе) + номерные города (Чехов-4), по которым нет данных
    - можно заполнить медианой (или средним) всех имеющихся значений численности менее 3 тыс.

- структура адресов из Mongo

In [33]:
client = MongoClient('localhost', 27017)
mongobase = client.bo_nalog
collection = mongobase['bonalogru']

In [34]:
# таблица с адресами - чтобы посмотреть соотношение
addresses = pd.DataFrame.from_records(collection.find({}, {'region': 1, 'district': 1, 'city': 1,
                                                           'settlement': 1}))

In [35]:
addresses.head(3)

,_id,region,district,city,settlement
0,3447001553,ВОЛГОГРАДСКАЯ,None,ВОЛГОГРАД,None
1,3447003470,ВОЛГОГРАДСКАЯ,None,ВОЛГОГРАД,None
2,3447003247,ВОЛГОГРАДСКАЯ,None,ВОЛГОГРАД,None


In [36]:
addresses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 726359 entries, 0 to 726358
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   _id         726359 non-null  object
 1   region      726343 non-null  object
 2   district    107696 non-null  object
 3   city        482674 non-null  object
 4   settlement  77759 non-null   object
dtypes: object(5)
memory usage: 27.7+ MB


In [38]:
# для тех, у кого нет региона, но есть город - регион можно восстановить
# но регион на самом деле не нужен, так как в postgresql в поле код региона нет пропусков
addresses.loc[addresses.region.isna()]

,_id,region,district,city,settlement
126136,3907027634,None,None,КАЛИНИНГРАД,None
138500,2312124742,None,None,КРАСНОДАР,None
138712,2312124887,None,None,КРАСНОДАР,None
168754,5406248869,None,None,НОВОСИБИРСК,None
223629,2311052083,None,None,КРАСНОДАР,None
277503,2310139006,None,None,КРАСНОДАР,None
325443,5404226193,None,None,НОВОСИБИРСК,None
394464,1654005143,None,None,КАЗАНЬ,None
489395,6315568302,None,None,САМАРА,None
495252,3702052848,None,None,ИВАНОВО,None


In [41]:
# district - очень мало значений
# а часть из них - города (как Ногинск или Одинцово ниже)
addresses.district.isna().sum()

618663

In [42]:
addresses.district.unique()

array([None, 'ОДИНЦОВО', 'Одинцово', ..., 'МИЛЬКОВСКИЙ', 'ЧАРЫШСКИЙ',
       'Хабарский'], dtype=object)

In [43]:
# но он по большей части бесполезен
addresses.loc[addresses.region.isna() & addresses.city.isna() & addresses.settlement.isna()]

,_id,region,district,city,settlement


- и в целом есть самые разные комбинации пропусков

In [44]:
addresses.loc[~addresses.settlement.isna() & addresses.city.isna()]

,_id,region,district,city,settlement
9,5032184021,МОСКОВСКАЯ,ОДИНЦОВО,None,ВНИИССОК
12,5032206081,МОСКОВСКАЯ,ОДИНЦОВО,None,ЛЕСНОЙ ГОРОДОК
13,5032197768,МОСКОВСКАЯ,ОДИНЦОВО,None,ШАРАПОВКА
26,5032206476,МОСКОВСКАЯ,ОДИНЦОВО,None,ЧАСЦЫ
27,5032194661,МОСКОВСКАЯ,ОДИНЦОВО,None,ШУЛЬГИНО
...,...,...,...,...,...
726352,2287004086,АЛТАЙСКИЙ,ЦЕЛИННЫЙ,None,ПОБЕДА
726353,2287005178,АЛТАЙСКИЙ,ЦЕЛИННЫЙ,None,ЦЕЛИННОЕ
726354,2287006781,АЛТАЙСКИЙ,Целинный,None,Ложкино
726355,2286002625,АЛТАЙСКИЙ,ХАБАРСКИЙ,None,ХАБАРЫ


In [45]:
# нет ни city ни district
addresses.loc[addresses.settlement.isna() & addresses.city.isna()]

,_id,region,district,city,settlement
7,5032200629,МОСКОВСКАЯ,ОДИНЦОВО,None,None
10,5032190642,МОСКОВСКАЯ,ОДИНЦОВО,None,None
14,5032185280,МОСКОВСКАЯ,ОДИНЦОВО,None,None
15,5032194816,МОСКВА,None,None,None
16,5032205666,МОСКОВСКАЯ,ОДИНЦОВО,None,None
...,...,...,...,...,...
726224,7731259097,МОСКВА,None,None,None
726225,7731252581,МОСКВА,None,None,None
726226,7731258086,МОСКВА,None,None,None
726227,7731232070,МОСКВА,None,None,None


In [46]:
# но большую часть можно заполнить - тк город указан в region
addresses.loc[addresses.settlement.isna() & addresses.city.isna()].region.unique()

array(['МОСКОВСКАЯ', 'МОСКВА', 'САНКТ-ПЕТЕРБУРГ', 'ЛЕНИНГРАДСКАЯ',
       'ХАБАРОВСКИЙ', 'СЕВАСТОПОЛЬ', 'САМАРСКАЯ', 'ОРЕНБУРГСКАЯ',
       'КОСТРОМСКАЯ', 'ИРКУТСКАЯ', 'ТАТАРСТАН', 'Севастополь',
       'УЛЬЯНОВСКАЯ', 'ПЕНЗЕНСКАЯ', 'НОВГОРОДСКАЯ', 'РОСТОВСКАЯ',
       'ПЕРМСКИЙ', 'УДМУРТСКАЯ', 'СВЕРДЛОВСКАЯ', 'ЛИПЕЦКАЯ', 'ИВАНОВСКАЯ',
       'БАШКОРТОСТАН', 'НОВОСИБИРСКАЯ', 'ПРИМОРСКИЙ', 'САРАТОВСКАЯ',
       'ТЮМЕНСКАЯ', 'ТАМБОВСКАЯ', 'ТВЕРСКАЯ', 'КАРЕЛИЯ', 'ЗАБАЙКАЛЬСКИЙ',
       'МУРМАНСКАЯ', 'ВОРОНЕЖСКАЯ', 'КРАСНОДАРСКИЙ', 'КРАСНОЯРСКИЙ',
       'БЕЛГОРОДСКАЯ', 'СТАВРОПОЛЬСКИЙ', 'НИЖЕГОРОДСКАЯ', 'КУРГАНСКАЯ',
       'ТОМСКАЯ', 'ТУЛЬСКАЯ', 'МАРИЙ ЭЛ', 'АСТРАХАНСКАЯ', 'КАЛУЖСКАЯ',
       'СМОЛЕНСКАЯ', 'ВЛАДИМИРСКАЯ', 'КАМЧАТСКИЙ', 'КОМИ', 'ЧЕЛЯБИНСКАЯ',
       'КАЛИНИНГРАДСКАЯ', 'БУРЯТИЯ', 'Санкт-Петербург', 'АЛТАЙ', 'Москва',
       'КРЫМ', 'КИРОВСКАЯ', 'САХАЛИНСКАЯ', 'РЯЗАНСКАЯ', 'Московская',
       'КУРСКАЯ', 'ПСКОВСКАЯ', 'АРХАНГЕЛЬСКАЯ',
       'ХАНТЫ-МАНСИЙСКИЙ АВТОНОМНЫЙ ОКРУГ

In [49]:
# Москва
addresses.loc[addresses.settlement.isna() & addresses.city.isna()].loc[addresses.region=='МОСКВА'].shape

(115002, 5)

In [50]:
# СПб
addresses.loc[addresses.settlement.isna() & addresses.city.isna()].loc[addresses.region=='САНКТ-ПЕТЕРБУРГ'].shape

(56662, 5)

In [51]:
addresses.loc[addresses.settlement.isna() & addresses.city.isna()].loc[addresses.region=='СЕВАСТОПОЛЬ'].shape

(2230, 5)

- для проверки - в xml действительно в поле Регион указан Город

      <Регион Наим="САНКТ-ПЕТЕРБУРГ" Тип="ГОРОД"/> 

- т.е. придётся заполнять пропуски и выбирать способ, как определить конечный пункт, по которому соединять с таблицей с численностью
- пока - уникальные city и settlement из Mongo

In [52]:
cities = collection.distinct('city')
cities = set(el.title() for el in cities[1:])

settlements = collection.distinct('settlement')
settlements = set(el.title() for el in settlements[1:])

len(cities), len(settlements)

(1184, 13526)

- данные переписи

In [54]:
data = pd.read_excel('../raw_data/tab-5_VPN-2020.xlsx', header=3)
population = data[['Unnamed: 0', 'Unnamed: 1']].rename(columns={'Unnamed: 0': 'locality', 'Unnamed: 1': 'population'})

In [55]:
# строки разного уровня + очень неединообразные наименования
# для города: город, г. г и т.д.
population

,locality,population
0,Российская Федерация,147182123
1,Городское население,110075322
2,Сельское население,37106801
3,Центральный федеральный округ,40334532
4,Городское население,33118003
...,...,...
27794,Сельское поселение Лаврентия,1420
27795,Сельское поселение Лорино,1426
27796,Сельское поселение Нешкан,712
27797,Сельское поселение Уэлен,685


In [57]:
# порядок важен - в городах сперва ищем город!
def get_population(name):
    val = population.loc[population['locality']\
          .str \
          .contains(rf'г. {name}\b|пгт {name}\b|село {name}\b|пос[ёе]лок {name}\b|п. {name}\b|[Сс]ельское поселение {name}\b|^{name}\b сельское поселение|Поселение {name}\b|[Дд]еревня {name}\b|г {name}\b|город {name}\b')]\
          .population.values
    return val[0] if val.any() else None

In [58]:
%%time
city_to_population = {key: get_population(key) for key in cities}

Wall time: 1min 51s


In [59]:
# не найдено значений для
for k, v in city_to_population.items():
    if not v:
        print(k, v)

Геническ None
Городской Округ Черноголовка None
Николаевск-На-Амуре None
Чехов-4 None
Добрунское Сельское Поселение None
Киссолово None
Васкелово None
Буденновск None
Новинка None
Павло-Слободское None
Славянск-На-Кубани None
Лошаковкое None
Усть-Лужская None
Мендсары None
Терсинское Мо None
Сольцы 2 None
Разинское None
Ростов-На-Дону None
Чехов-2 None
Чехов-6 None
Красноярское Мо None
Городское Поселение Снегири None
Алешино None
Энгельс-19 None
Сертолово-2 None
Мо Сельское Поселение Следневское None
Булатниковское None
Белохолуницкое Городское Поселение None
Энгельс-2 None
Зеленоград None
Климовск None
38 Км Выборгского Шоссе None
Посадников Остров None
Чехов-3 None
Керро None
Комсомольск-На-Амуре None
45 Км Автодороги Санкт-Петербург-Псков None
26 Км Приозерского Шоссе None
Одинцово-10 None
Камень-На-Оби None
Калач-На-Дону None
Лесковское None
Новопушкинское Мо None
Новохоперск None
Широкобуеракское Мо None
Мертуть None
Королев None
Боровск-1 None


In [ ]:
# часть можно всё-таки найти, если очистить данные
# словарь для замены в датасете: - надо будет обновить, так как данные ещё добавляются
city_replace = {
    'Королев': 'Королёв',
    'Белохолуницкое Городское Поселение': 'Белохолуницкое',
    'Мо Сельское Поселение Следневское': 'Следневское'
    'Камень-На-Оби': 'Камень-на-Оби',
    'Широкобуеракское Мо': 'Широкобуеракское',
    'Новопушкинское Мо': 'Новопушкинское',
    'Ростов-На-Дону': 'Ростов-на-Дону',
    'Сольцы 2': 'Сольцы',
    'Буденновск': 'Будённовск',
    'Городское Поселение Снегири': 'Снегири',
    'Калач-На-Дону': 'Калач-на-Дону',
    'Славянск-На-Кубани': 'Славянск-на-Кубани',
    'Терсинское Мо': 'Терсинское',
    'Николаевск-На-Амуре': 'Николаевск-на-Амуре',
    'Комсомольск-На-Амуре': 'Комсомольск-на-Амуре',
}

- для settlement - можно искать таким же образом (только совпадений намного меньше и поиск заметно дольше) - из 11 тыс уникальных всего около 2 тыс значений численности
- большая часть из этих None - менее 3 тыс населения!

### предварительный отбор полей, пропуски и типы данных в таблицах

#### - из MongoDB bo_nalog.bonalogru

- карточка организации + отчёты

In [61]:
client = MongoClient('localhost', 27017)
mongobase = client.bo_nalog
collection = mongobase['bonalogru']

In [62]:
# поля снова указываю явно - проще выключить/включить
# уже предваительно отобранные
fields = {
    '_id': 1,
#     'id_org': 1,  # id карточки организации
    'shortName': 1,  # только чтобы проще сориентироваться - потом убрать!
    'ogrn': 1,
#     'index': 1,  # почтовый индекс, можно что-нибудь привязать, но мне не нужен пока
    'region': 1,  # здесь достаточно много городов в к-ве региона (Москва, СПб...)
#     'district': 1,
    'city': 1,  # здесь достаточно много поселений/посёлков, пгт и др
    'settlement': 1,
#     'okved': 1,  # просто на всякий случай, скорее всего убрать, ага все None
    'okved2': 1,
    'okopf': 1,
    'statusCode': 1,  # - даже полезно!, мб целью?
    'registrationDate': 1,
    'authorizedCapital': 1,
#     'report_periods': 1,
    # АКТИВ
    # I Внеоборотные активы
    # Нематериальные активы
#     '2021_1110': 1,
#     '2020_1110': 1,
#     '2019_1110': 1,
    # Результаты исследований и разработок (около 3 тыс на 650 тыс)
#     '2021_1120': 1,  # а здесь у меня записано поле 2021_1220, но они всё равно не нужны
#     '2020_1120': 1,  # это верно
#     '2019_1120': 1,  # и это верно
    # Нематериальные поисковые активы
#     '2021_1130': 1,
#     '2020_1130': 1,
#     '2019_1130': 1,
    # Материальные поисковые активы
#     '2021_1140': 1,
#     '2020_1140': 1,
#     '2019_1140': 1,
    # Основные средства = Материальные внеоборотные активы - есть в упрощённой форме
    '2021_1150': 1,
    '2020_1150': 1,
    '2019_1150': 1,
    # Доходные вложения в материальные ценности
#     '2021_1160': 1,
#     '2020_1160': 1,
#     '2019_1160': 1,
    # Финансовые вложения = Нематериальные, финансовые и другие внеоборотные активы
    # - есть в упрощённой форме, хотя значений мало
    '2021_1170': 1,
    '2020_1170': 1,
    '2019_1170': 1,
    # Отложенные налоговые активы
#     '2021_1180': 1,
#     '2020_1180': 1,
#     '2019_1180': 1,
    # Прочие внеоборотные активы
#     '2021_1190': 1,
#     '2020_1190': 1,
#     '2019_1190': 1,
    # Итого по разделу I - их довольно много (145385, но для упрощённой формы нет
    # те можно использовать, если отбирать организации по типу)
#     '2021_1100': 1,
#     '2020_1100': 1,
#     '2019_1100': 1,
    # II Оборотные активы
    # Запасы - есть в упрощённой форме
    '2021_1210': 1,
    '2020_1210': 1,
    '2019_1210': 1,
    # Налог на добавленную стоимость по приобретённым ценностям
#     '2021_1220': 1,  # здесь у меня ошибка, поле 2021_1220 записала в 2021_1120
#     '2020_1220': 1,  # около 50 тыс на 650 тыс
#     '2019_1220': 1,
    # Дебиторская задолженность = Финансовые и другие оборотные активы?
    # долговые обязательства со стороны контрагентов, партнеров, клиентов или работников
    # есть в упрощённой форме
    '2021_1230': 1,
    '2020_1230': 1,
    '2019_1230': 1,
    # Финансовые вложения (за исключением денежных эквивалентов)
#     '2021_1240': 1,
#     '2020_1240': 1,
#     '2019_1240': 1,
    # Денежные средства и денежные эквиваленты - есть в упрощённой форме
    '2021_1250': 1,
    '2020_1250': 1,
    '2019_1250': 1,
    # Прочие оборотные активы
#     '2021_1260': 1,
#     '2020_1260': 1,
#     '2019_1260': 1,
    # Итого по разделу II - в упрощённой форме нет, но их довольно много (ок 250 тыс)
#     '2021_1200': 1,
#     '2020_1200': 1,
#     '2019_1200': 1,
    # Баланс по АКТИВЫ - есть в упрощённой форме, пропусков нет!
    '2021_1600': 1,
    '2020_1600': 1,
    '2019_1600': 1,
    # ПАССИВ
    # III Капитал и резервы
    # Уставный капитал (складочный капитал, уставный фонд, вклады товарищей)
    # снова довольно много
#     '2021_1310': 1,
#     '2020_1310': 1,
#     '2019_1310': 1,
    # Собственные акции, выкупленные у акционеров
    # только для тех, у кого есть акции
#     '2021_1320': 1,
#     '2020_1320': 1,
#     '2019_1320': 1,
    # Переоценка внеоборотных активов
#     '2021_1340': 1,
#     '2020_1340': 1,
#     '2019_1340': 1,
    # Добавочный капитал (без переоценки) = Целевые средства
    # есть в упрощённой форме, но очень мало значений (17 тыс)
    '2021_1350': 1,
    '2020_1350': 1,
    '2019_1350': 1,
    # Резервный капитал = Фонд недвижимого и особо ценного движимого имущества и иные 
    # целевые фонды - то же самое, есть в упрощенной, около 15 тыс
    '2021_1360': 1,
    '2020_1360': 1,
    '2019_1360': 1,
    # Нераспределенная прибыль (непокрытый убыток) - нет в упрощённой, но довольно много значений
#     '2021_1370': 1,
#     '2020_1370': 1,
#     '2019_1370': 1,
    # Итого по разделу III = Капитал и резервы - есть в упрощённой, много (631996)
    '2021_1300': 1,
    '2020_1300': 1,
    '2019_1300': 1,
    # IV. Долгосрочные обязательства
    # Заемные средства = Краткосрочные заемные средства, есть в упрощённой, 204 тыс, неплохо
    # отсутствие, наверное = 0?
    # долгосрочные заёмные средства должны были быть под кодом 1410 и 1450 - но у меня их нет
    # (видимо, разные формы...)
    # ну и чёрт с ним, выключаем, главное, что баланс собран
    # ага, куда-то делся целыйблок кодов с 1400
#     '2021_1510': 1,
#     '2020_1510': 1,
#     '2019_1510': 1,
    # Кредиторская задолженность - есть в упрощённой, много 550 тыс, мб целью!
    '2021_1520': 1,
    '2020_1520': 1,
    '2019_1520': 1,
    # Доходы будущих периодов
#     '2021_1530': 1,
#     '2020_1530': 1,
#     '2019_1530': 1,
    # Оценочные обязательства
#     '2021_1540': 1,
#     '2020_1540': 1,
#     '2019_1540': 1,
    # Прочие обязательства = Другие краткосрочные обязательства - есть в упрощённой
#     '2021_1550': 1,
#     '2020_1550': 1,
#     '2019_1550': 1,
    # Итого по разделу V - нет в упрощённой, около 200 тыс
#     '2021_1500': 1,
#     '2020_1500': 1,
#     '2019_1500': 1,
    # Баланс по ПАССИВЫ - есть в упрощённой, нет пропусков!
    '2021_1700': 1,
    '2020_1700': 1,
    '2019_1700': 1,
    # fin_res
    # Выручка - есть в упрощённой, 500 тыс
    '2021_2110': 1,
    '2020_2110': 1,
    # Себестоимость продаж = Расходы по обычной деятельности - есть в упрощённой, 500 тыс
    '2021_2120': 1,
    '2020_2120': 1,
    # Валовая прибыль (убыток) - нет в упрощённой, 200 тыс
#     '2021_2100': 1,
#     '2020_2100': 1,
    # Управленческие расходы - нет в упрощённой, 100 тыс
#     '2021_2220': 1,
#     '2020_2220': 1,
    # Прибыль (убыток) от продаж - нет в упрощённой, 200 тыс
#     '2021_2200': 1,
#     '2020_2200': 1,
    # Проценты к уплате - есть в упрощённой, 100 тыс
    '2021_2330': 1,
    '2020_2330': 1,
    # Прочие доходы - есть в упрощённой, 300 тыс
    '2021_2340': 1,
    '2020_2340': 1,
    # Прочие расходы - есть в упрощённой, 500 тыс
    '2021_2350': 1,
    '2020_2350': 1,
    # Прибыль (убыток) до налогообложения - нет в упрощённой, 200 тыс
#     '2021_2300': 1,
#     '2020_2300': 1,
    # Налог на прибыль (доходы) - есть в упрощённой, 450 тыс
    '2021_2410': 1,
    '2020_2410': 1,
    # Чистая прибыль (убыток) - есть в упрощённой - 560 тыс
    '2021_2400': 1,
    '2020_2400': 1,
    # Совокупный финансовый результат периода (не для упрощённой формы) - 200 тыс
#     '2021_2500': 1,
#     '2020_2500': 1,
}

In [63]:
len(fields)

58

In [64]:
mongo_data = pd.DataFrame.from_records(collection.find({}, fields))

In [66]:
mongo_data.head(3)

,_id,shortName,ogrn,region,city,settlement,okved2,okopf,statusCode,registrationDate,...,2021_2330,2020_2330,2021_2340,2020_2340,2021_2350,2020_2350,2021_2410,2020_2410,2021_2400,2020_2400
0,3447001553,"ООО ""МОДЕРН-БОН""",1023404293505,ВОЛГОГРАДСКАЯ,ВОЛГОГРАД,None,93.19,12300.0,ACTIVE,1992-12-14,...,0.0,0.0,2100.0,0.0,714.0,0.0,138.0,0.0,1270.0,0.0
1,3447003470,"ООО ""МАРКО""",1023404287829,ВОЛГОГРАДСКАЯ,ВОЛГОГРАД,None,68.20.2,12300.0,ACTIVE,2002-05-20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-29.0,30.0
2,3447003247,"ООО ""ОКСИ""",1023444287855,ВОЛГОГРАДСКАЯ,ВОЛГОГРАД,None,20.11,12300.0,ACTIVE,1998-10-01,...,NaN,NaN,163.0,NaN,410.0,83.0,152.0,113.0,1099.0,691.0


In [65]:
mongo_data.info(verbose=True, show_counts=True, memory_usage=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 726359 entries, 0 to 726358
Data columns (total 58 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   _id                726359 non-null  object 
 1   shortName          726359 non-null  object 
 2   ogrn               726359 non-null  object 
 3   region             726343 non-null  object 
 4   city               482674 non-null  object 
 5   settlement         77759 non-null   object 
 6   okved2             726002 non-null  object 
 7   okopf              710454 non-null  float64
 8   statusCode         726359 non-null  object 
 9   registrationDate   726358 non-null  object 
 10  authorizedCapital  698317 non-null  float64
 11  2021_1150          315778 non-null  float64
 12  2020_1150          260541 non-null  float64
 13  2019_1150          240928 non-null  float64
 14  2021_1170          84083 non-null   float64
 15  2020_1170          58702 non-null   float64
 16  20

- **statusCode** неожиданно полезное поле, может быть целью, хотя и есть достаточно сильный дисбаланс

In [67]:
mongo_data.statusCode.value_counts()

ACTIVE                  710682
LIQUIDATION_STAGE        12184
INACTIVE                  1927
REORGANIZATION_STAGE      1566
Name: statusCode, dtype: int64

- все даты статуса LIQUIDATION_STAGE и INACTIVE - в промежутке 2022 - 2023 гг, 
- для всех из них есть отчётся за 2021 г - те статус был ACTIVE (иначе отчёт не публикуется)
- те можно, например, предсказывать смену статуса в ближайшие два года?

- ещё раз, на всякий случай, чищу inn (так как первые части собирала без очистки)

In [68]:
mongo_data._id = mongo_data._id.apply(lambda x: x.strip())
mongo_data._id.nunique(), mongo_data._id.size

(726359, 726359)

- **okopf** распознал как float
- это не играет большой роли, но если понадобится сджойнить со справочником кодов - может быть неуодбно
- плюс есть немного пропусков

In [69]:
mongo_data.okopf.unique()

array([12300., 12247., 14200., 12267., 20101., 20102., 20100., 20107.,
       20200., 70401., 20112., 12200., 14100., 15300., 20110., 20109.,
       14154., 14153., 11064., 14000., 20111., 20116., 20700., 11051.,
       20115.,    nan, 20202., 71400., 20716., 20702., 14155., 70400.,
       65243., 75500., 71500., 20614., 20210., 21100., 20615., 20105.,
       20606., 70403., 20616., 21200., 20104., 20201., 20610., 75505.,
       20600., 75404., 20108., 20103., 20217., 20605.])

In [70]:
# пока просто присвоим 0 для None, приведём к int (чтобы избавиться от нулей) 
# и к object (тк категория)
mongo_data.okopf = mongo_data.okopf.fillna(0).astype('int32').astype('object')

- немного явных выбросов

In [73]:
mongo_data.authorizedCapital.describe()

count    6.983170e+05
mean     5.857532e+06
std      2.335524e+09
min      0.000000e+00
25%      1.000000e+04
50%      1.000000e+04
75%      2.000000e+04
max      1.900100e+12
Name: authorizedCapital, dtype: float64

In [72]:
# Организация похорон и представление связанных с ними услуг, 1,9 трлн ₽ уставного капитала
# финансовых результатов нет, все бух. отчёты стабильны - одни и те же цифры все 3 года
# статус ACTIVE
mongo_data.loc[mongo_data.authorizedCapital == 1.900100e+12]

,_id,shortName,ogrn,region,city,settlement,okved2,okopf,statusCode,registrationDate,...,2021_2330,2020_2330,2021_2340,2020_2340,2021_2350,2020_2350,2021_2410,2020_2410,2021_2400,2020_2400
96403,6679020093,"ООО ""ВИТА-ПРОМ""",1126679022953,СВЕРДЛОВСКАЯ,ЕКАТЕРИНБУРГ,None,96.03,12300,ACTIVE,2012-08-23,...,NaN,NaN,0.0,7170.0,NaN,NaN,NaN,NaN,0.0,7170.0


- c адресом придётся работать достаточно подробно

### - reestr_msp

In [76]:
connect = psycopg2.connect(dbname="fns_data", user="user", password='new_password', host="localhost", port=5432)

In [79]:
connect.close()

In [77]:
columns = [
    'inn', 
    'n_employees', 
    'is_soc_enterprise', 
    'newly_establ', 
    'subj_cat', 
    'subj_form', 
    'inclusion_date',  # отличается от registration_date! - сравнить!
    'region_code', 
    'n_okovd_add',  
#     'okovd_add_vers',  # только убедиться, что везде одно значение! - везде, где есть, одно
#     'okovd_add_code',  # можно придумать много интересных признаков
    'n_licenses',
    'n_products',
#     'products_code'  # то же самое
    'is_high_tech',
    'n_contracts',
    'n_agreements'
]

In [78]:
%%time
reestr_msp = pd.read_sql(sql.SQL('SELECT {} FROM reestr_msp;')\
                            .format(sql.SQL(',').join(map(sql.Identifier, columns))), connect)

Wall time: 15.4 s


In [80]:
reestr_msp.head()

,inn,n_employees,is_soc_enterprise,newly_establ,subj_cat,subj_form,inclusion_date,region_code,n_okovd_add,n_licenses,n_products,is_high_tech,n_contracts,n_agreements
0,3447001553,1.0,0,0,1,1,2016-08-01,34,29,0,0,0,0,0
1,3447003470,2.0,0,0,1,1,2016-08-01,34,6,0,0,0,0,0
2,3447003247,18.0,0,0,1,1,2016-08-01,34,24,1,0,0,0,0
3,3447003416,165.0,0,0,3,1,2018-08-10,34,9,2,0,0,0,0
4,3447002444,NaN,0,0,1,1,2016-08-01,34,0,0,0,0,0,0


In [82]:
reestr_msp.info(verbose=True, show_counts=True, memory_usage=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2332299 entries, 0 to 2332298
Data columns (total 14 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   inn                2332299 non-null  object 
 1   n_employees        1938849 non-null  float64
 2   is_soc_enterprise  2332299 non-null  int64  
 3   newly_establ       2332299 non-null  int64  
 4   subj_cat           2332299 non-null  int64  
 5   subj_form          2332299 non-null  int64  
 6   inclusion_date     2332299 non-null  object 
 7   region_code        2332299 non-null  int64  
 8   n_okovd_add        2332299 non-null  int64  
 9   n_licenses         2332299 non-null  int64  
 10  n_products         2332299 non-null  int64  
 11  is_high_tech       2332299 non-null  int64  
 12  n_contracts        2332299 non-null  int64  
 13  n_agreements       2332299 non-null  int64  
dtypes: float64(1), int64(11), object(2)
memory usage: 249.1+ MB


- очень немного пропусков
- можно заметно уменьшить, приведя типы

In [83]:
# есть странные значения численности работников
reestr_msp.n_employees.min(), reestr_msp.n_employees.max()

(-1.0, 8617.0)

In [84]:
# Деятельность зрелищно-развлекательная прочая
# во всех отчётах - нули
reestr_msp.loc[reestr_msp.n_employees == -1]

,inn,n_employees,is_soc_enterprise,newly_establ,subj_cat,subj_form,inclusion_date,region_code,n_okovd_add,n_licenses,n_products,is_high_tech,n_contracts,n_agreements
133484,5835136064,-1.0,0,0,1,1,2022-07-10,58,38,0,0,0,0,0


In [88]:
# здесь пока ещё нет
mongo_data.loc[mongo_data._id == '5835136064']

,_id,shortName,ogrn,region,city,settlement,okved2,okopf,statusCode,registrationDate,...,2021_2330,2020_2330,2021_2340,2020_2340,2021_2350,2020_2350,2021_2410,2020_2410,2021_2400,2020_2400


In [89]:
# приведу к int16, None временно закодирую как -2
reestr_msp.n_employees = reestr_msp.n_employees.fillna(-2).astype('int16')

In [90]:
# максимальные значения
reestr_msp.region_code.max(), reestr_msp.n_okovd_add.max(), reestr_msp.n_licenses.max(), reestr_msp.n_products.max(), reestr_msp.n_contracts.max(), reestr_msp.n_agreements.max()

(95, 2468, 251, 70, 1084, 401)

- 2468 - это довольно много для дополнительных видов деятельности (n_okovd_add)

In [92]:
# region_code - категория
reestr_msp.region_code = reestr_msp.region_code.astype('object')

In [93]:
columns_8 = ['is_soc_enterprise', 'newly_establ', 'subj_cat', 'subj_form', 'region_code', 
           'n_products', 'is_high_tech']
columns_16 = ['n_okovd_add', 'n_licenses', 'n_contracts', 'n_agreements']
for col in columns_8:
    reestr_msp[col] = reestr_msp[col].astype('int8')
for col in columns_16:
    reestr_msp[col] = reestr_msp[col].astype('int16')

In [94]:
reestr_msp.info(verbose=True, show_counts=True, memory_usage=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2332299 entries, 0 to 2332298
Data columns (total 14 columns):
 #   Column             Non-Null Count    Dtype 
---  ------             --------------    ----- 
 0   inn                2332299 non-null  object
 1   n_employees        2332299 non-null  int16 
 2   is_soc_enterprise  2332299 non-null  int8  
 3   newly_establ       2332299 non-null  int8  
 4   subj_cat           2332299 non-null  int8  
 5   subj_form          2332299 non-null  int8  
 6   inclusion_date     2332299 non-null  object
 7   region_code        2332299 non-null  int8  
 8   n_okovd_add        2332299 non-null  int16 
 9   n_licenses         2332299 non-null  int16 
 10  n_products         2332299 non-null  int8  
 11  is_high_tech       2332299 non-null  int8  
 12  n_contracts        2332299 non-null  int16 
 13  n_agreements       2332299 non-null  int16 
dtypes: int16(5), int8(7), object(2)
memory usage: 73.4+ MB


### - support_msp

- на 1 инн - несколько строк, строка = акт поддержки
- есть дата, есть вид поддержки, есть единица поддержки

In [95]:
connect = psycopg2.connect(dbname="fns_data", user="user", password='', host="localhost", port=5432)

In [98]:
connect.close()

In [96]:
columns = [
    'inn', 
    'support_date', 
    'is_breach', 
    'form_code', 
    'support_volume', 
    'support_unit', 
]

In [97]:
%%time
support_msp = pd.read_sql(sql.SQL('SELECT {} FROM support_msp;')\
                            .format(sql.SQL(',').join(map(sql.Identifier, columns))), connect)

Wall time: 5.27 s


In [99]:
# пропусков нет
support_msp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1170650 entries, 0 to 1170649
Data columns (total 6 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   inn             1170650 non-null  object 
 1   support_date    1170650 non-null  object 
 2   is_breach       1170650 non-null  int64  
 3   form_code       1170650 non-null  object 
 4   support_volume  1170650 non-null  float64
 5   support_unit    1170650 non-null  int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 53.6+ MB


In [100]:
# это флаг - было ли нарушение условий
support_msp.is_breach.max()

1

In [102]:
support_msp.is_breach = support_msp.is_breach.astype('int8')

- единица поддержки - это категория, но скорее всего не понадобится, тк здесь всего 2 единицы: 1 - деньги и 4 - кажется, проценты

In [103]:
support_msp.support_unit = support_msp.support_unit.astype('object')

In [104]:
support_msp.support_unit.unique()

array([1, 4], dtype=object)

In [105]:
support_msp.form_code.unique()

array(['0103', '0104', '0102', '0105', '0101'], dtype=object)

    - субсидии и гранты, КодВид="0103"
    - неизвестно, насколько корректно суммировать гарантии и поручительство ="0102", ед. поддержки тоже 1 (деньги)
    - финансирование на возвратной основе КодВид="0104" ед поддержки 4 (проценты?)
    - Финансовая аренда (лизинг) = 0105, сразу две ед - 1 и 4
    - Инвестиции в капитал = 0101, ед поддержки 1

- немного агрегатов (сложение для ед поддержки 4 - финансирование на возвратной основе, видимо, некорректно)
- все остальные коды - единица = 1, за искл 0105 - Финансовая аренда (лизинг), там единицы и 1 и 4, те единицу 1 можно складывать,

In [106]:
# ага, кажется, действительно проценты
support_msp.loc[support_msp.form_code == '0104', 'support_volume'].max(), support_msp.loc[support_msp.form_code == '0104', 'support_volume'].min()

(100.0, 0.0)

In [109]:
agg_data = pd.pivot_table(support_msp.loc[support_msp.form_code != '0104'], values='support_volume', 
               index='inn', columns=['form_code'], aggfunc=sum, )#fill_value=0)

In [110]:
agg_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 278610 entries, 0100000156 to 9901088889
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   0101    114 non-null     float64
 1   0102    50059 non-null   float64
 2   0103    240428 non-null  float64
 3   0105    845 non-null     float64
dtypes: float64(4)
memory usage: 10.6+ MB


In [113]:
# полторы тысячи с нарушениями условий поддержки - можно оставить как флаг?
support_msp.loc[(support_msp.is_breach == 1) & (support_msp.form_code != '0104')]

,inn,support_date,is_breach,form_code,support_volume,support_unit
12483,4213005280,2022-06-02,1,0103,103024.04,1
12485,4213005280,2021-07-22,1,0103,5159768.53,1
12486,4213005280,2021-10-19,1,0103,224988.28,1
12488,4213005280,2021-12-13,1,0103,2198451.59,1
12489,4213005280,2021-10-07,1,0103,468130.00,1
...,...,...,...,...,...,...
1154338,4205393624,2021-07-19,1,0103,1005061.15,1
1154340,4205393624,2022-06-03,1,0103,1059607.02,1
1154342,4205393624,2021-11-29,1,0103,444.78,1
1154343,4205393624,2021-05-20,1,0103,514634.57,1


- нужно учесть и дату - взять поддержку за определённый период!

In [115]:
pd.to_datetime(support_msp.support_date).dt.year.unique()

array([2020, 2021, 2022, 2019, 2023], dtype=int64)

### - debt

- эти данные могут быть целью

In [116]:
connect = psycopg2.connect(dbname="fns_data", user="user", password='', host="localhost", port=5432)

In [119]:
connect.close()

In [117]:
columns = [
    'inn', 
#     'status_date',  # [datetime.date(2021, 12, 31)]
    'arrears', 
    'penalties', 
    'fines', 
]

In [118]:
debt = pd.read_sql(sql.SQL('SELECT {} FROM debt;')\
                            .format(sql.SQL(',').join(map(sql.Identifier, columns))), connect)

- здесь уже аггрегированна информация, дата везде одна! - это дата, на которую верно состояние

In [120]:
# пропусков нет
debt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1080232 entries, 0 to 1080231
Data columns (total 4 columns):
 #   Column     Non-Null Count    Dtype  
---  ------     --------------    -----  
 0   inn        1080232 non-null  object 
 1   arrears    1080232 non-null  float64
 2   penalties  1080232 non-null  float64
 3   fines      1080232 non-null  float64
dtypes: float64(3), object(1)
memory usage: 33.0+ MB


### - tax_offense

- здесь налоговые нарушения - тоже может быть целью

In [121]:
connect = psycopg2.connect(dbname="fns_data", user="user", password='', host="localhost", port=5432)

In [124]:
connect.close()

In [122]:
columns = [
    'inn', 
#     'offense_date',  # одна для всех - [datetime.date(2021, 12, 31)]
    'fine_amount', 
]

In [123]:
tax_offense = pd.read_sql(sql.SQL('SELECT {} FROM tax_offense;')\
                            .format(sql.SQL(',').join(map(sql.Identifier, columns))), connect)

In [125]:
# пропусков нет
tax_offense.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129827 entries, 0 to 129826
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   inn          129827 non-null  object 
 1   fine_amount  129827 non-null  float64
dtypes: float64(1), object(1)
memory usage: 2.0+ MB


- довольно большая часть данных будет коррелирована (так как есть обобщённые столбцы и столбцы, конкретизирующие обобщённую информацию) - те придётся ещё отбирать
- также пока не выбрала период

    - можно, например, взять данные за 2021 год и предсказывать статус организации на 2022 и 2023 год
    - можно взять за 2020 и предсказывать задолженность, баланс или финансовые нарушения на 2021 год...
    - и т.д.

- данные обновляются достаточно редко (для большей части орагнизаций - только ежегодные публичные отчёты) + время выкладки
- я не учитываю коррекции и поправки данных - а для них предусмотрены свои формы